## Time for Salt bridge Interaction Calculation using Numpy Vectorized Array Implementation

In [ ]:
from pyspark.sql import SparkSession
from mmtfPyspark.io import mmtfReader
from mmtfPyspark.interactions import InteractionExtractor, InteractionFilter
from mmtfPyspark.webfilters import Pisces
import papermill as pm
import time

### Run benchmark

In [ ]:
t0 = time.time()
spark = SparkSession.builder.master("local[" + str(cores) + "]").appName("Saltbridges").getOrCreate()

t1 = time.time()
structures = mmtfReader.read_sequence_file(path)
structures = structures.filter(lambda s: s[1].num_models == 1)
                               
salt_bridge = InteractionFilter(distanceCutoff=3.5)
salt_bridge.set_query_groups(True, ['ASP', 'GLU'])
salt_bridge.set_query_atom_names(True, ['OD1', 'OD2', 'OE1', 'OE2'])
salt_bridge.set_target_groups(True, ['ARG', 'LYS', 'HIS'])
salt_bridge.set_target_atom_names(True, ['NH1', 'NH2', 'NZ', 'ND1', 'NE2'])

interactions = InteractionExtractor.get_polymer_interactions(structures, salt_bridge)
count = interactions.count()

t2 = time.time()

spark.stop()
t3 = time.time()

### Record results

In [ ]:
pm.record("saltbridges_total", t3 - t0)
pm.record("saltbridges", t2 - t1)
pm.record("count", count)